# Exploring the UEFA players kaggle dataset 
### /kaggle/input/uefa-euro-2024-players/euro2024_players.csv

In [4]:
import numpy as np 
import pandas as pd

In [5]:
df_euro = pd.read_csv("euro2024_players.csv")

In [6]:
df_euro.head()

,Name,Position,Age,Club,Height,Foot,Caps,Goals,MarketValue,Country
0,Marc-André ter Stegen,Goalkeeper,32,FC Barcelona,187,right,40,0,28000000,Germany
1,Manuel Neuer,Goalkeeper,38,Bayern Munich,193,right,119,0,4000000,Germany
2,Oliver Baumann,Goalkeeper,34,TSG 1899 Hoffenheim,187,right,0,0,3000000,Germany
3,Nico Schlotterbeck,Centre-Back,24,Borussia Dortmund,191,left,12,0,40000000,Germany
4,Jonathan Tah,Centre-Back,28,Bayer 04 Leverkusen,195,right,25,0,30000000,Germany


## What are the top 5 common names of players at EURO 2024?

In [7]:
df_palyersName = df_euro['Name'].str.split(' ', n=1, expand=True)
df_palyersName = df_palyersName.rename(columns={0:'Name', 1:'Surname'})
top_5_names = df_palyersName['Name'].value_counts().head(5)
top_5_names

Name
David     9
Luka      7
Giorgi    7
Tomas     6
Thomas    5
Name: count, dtype: int64

The fast result is unexpected: **David, Luka, and Giorgi**? Why? Where is Alexander?
However, **Tomas ~ Thomas**

## Let's investigate how many causes like **Tomas ~ Thomas** are in df

In [8]:
#import lib to compare names
from thefuzz import fuzz

In [9]:
names_unique = df_palyersName['Name'].unique()

In [11]:
len(names_unique) # count of unique names in dataset

467

In [12]:
names_unique

array(['Marc-André', 'Manuel', 'Oliver', 'Nico', 'Jonathan', 'Antonio',
       'Waldemar', 'Robin', 'David', 'Maximilian', 'Joshua', 'Benjamin',
       'Aleksandar', 'Robert', 'İlkay', 'Toni', 'Pascal', 'Florian',
       'Jamal', 'Kai', 'Chris', 'Leroy', 'Thomas', 'Deniz', 'Niclas',
       'Angus', 'Liam', 'Zander', 'Scott', 'Jack', 'Ryan', 'Grant',
       'Andrew', 'Kieran', 'Greg', 'Anthony', 'Ross', 'Billy', 'John',
       'Callum', 'Stuart', 'Kenny', 'Lewis', 'James', 'Ché', 'Lawrence',
       'Tommy', 'Péter', 'Dénes', 'Willi', 'Attila', 'Márton', 'Botond',
       'Ádám', 'Milos', 'Zsolt', 'Bendegúz', 'Endre', 'Loïc', 'Dominik',
       'András', 'Mihály', 'Dániel', 'László', 'Krisztofer', 'Roland',
       'Kevin', 'Barnabás', 'Martin', 'Gregor', 'Yann', 'Yvon', 'Fabian',
       'Leonidas', 'Cédric', 'Ricardo', 'Silvan', 'Denis', 'Granit',
       'Ardon', 'Michel', 'Remo', 'Vincent', 'Xherdan', 'Noah', 'Steven',
       'Dan', 'Rubén', 'Renato', 'Breel', 'Zeki', 'Kwadwo', 'Unai',
  

In [13]:
#Let's create a cartesian product for the list of names, to compare each other
df_names_vectors = pd.DataFrame(np.array(np.meshgrid(names_unique, names_unique)).T.reshape(-1,2), columns=['Name1', 'Name2'])

In [17]:
#Function to compare names by rows in the cartesian product dataframe
def df_row_similarity(row):
    return fuzz.ratio(row["Name1"], row["Name2"])

#adding similarity column as a result to every row
df_names_vectors["Similarity"] = df_names_vectors.apply(df_row_similarity, axis=1)

In [18]:
#if names in the row are equals to mark as Equality = True for futher filtering
df_names_vectors["Equality"] = df_names_vectors["Name1"] == df_names_vectors["Name2"]

In [19]:
#result table
df_names_vectors

,Name1,Name2,Similarity,Equality
0,Marc-André,Marc-André,100,True
1,Marc-André,Manuel,38,False
2,Marc-André,Oliver,12,False
3,Marc-André,Nico,14,False
4,Marc-André,Jonathan,22,False
...,...,...,...,...
218084,Mojmír,Vladimír,43,False
218085,Mojmír,Pavel,0,False
218086,Mojmír,Antonín,31,False
218087,Mojmír,Vaclav,0,False


In [20]:
#Print sample table to understand the Similarity level based on "Alex" name
#filter out ["Equality"] != True and sorted by ['Similarity'] desc
mask_equality_out =  df_names_vectors["Equality"] != True 
mask_alex_included = df_names_vectors["Name1"].str.contains('alex',case=False) 
df_alex = df_names_vectors[mask_equality_out & mask_alex_included].sort_values(by = ['Similarity'], ascending = False)
df_alex.head(30)

,Name1,Name2,Similarity,Equality
175347,Alexandru,Alexander,89,False
104049,Alexander,Alexandru,89,False
103771,Alexander,Alejandro,78,False
175222,Alexandru,Alejandro,78,False
73412,Alex,Álex,75,False
103832,Alexander,Alessandro,74,False
103686,Alexander,Aleksandar,74,False
175137,Alexandru,Aleksandar,74,False
175283,Alexandru,Alessandro,74,False
103904,Alexander,Anders,67,False


In [23]:
#Based on "Alex" name distribution sample Let's take 74 as a level of Similarity
mask_similarity = df_names_vectors["Similarity"] >= 74

In [24]:
#deleting sample data
del df_alex, mask_alex_included

In [27]:
#We should also consider options with lower Similarity level such as Alex - Alexander (Similarity 62)

In [28]:
#Let's add another parameter as "Inclusion" when longer name contains short name
def df_row_name_inclusion(row):
    return row["Name2"] in row["Name1"]

df_names_vectors["Name_inclusion"] = df_names_vectors.apply(df_row_name_inclusion, axis=1)

In [29]:
#create the new mask for inclusion
mask_inclusion = df_names_vectors["Name_inclusion"] == True

In [31]:
#Check if the result of inclusion contains data as expected
df_names_vectors[mask_equality_out & mask_inclusion & (df_names_vectors["Name_inclusion"] == True)].head(10)

,Name1,Name2,Similarity,Equality,Name_inclusion
98,Marc-André,Marc,57,False,True
2052,Jonathan,Jon,55,False,True
4535,Maximilian,Maxim,67,False,True
12789,Zander,Zan,67,False,True
32257,Gregor,Greg,80,False,True
34451,Leonidas,Leo,55,False,True
43984,Dani,Dan,86,False,True
46319,Daniel,Dan,67,False,True
46327,Daniel,Dani,80,False,True
48872,Mikel,Mike,89,False,True


In [32]:
#create df with list of names we are considering as equal
df_equal_names = df_names_vectors[mask_equality_out & (mask_similarity | mask_inclusion)][["Name1", "Name2"]]

In [34]:
df_equal_names

,Name1,Name2
98,Marc-André,Marc
1260,Oliver,Olivier
1569,Nico,Nicolò
1662,Nico,Nicola
2052,Jonathan,Jon
...,...,...
210249,Danilo,Daniel
210973,Matheus,Matus
212588,Pedro,Pedri
213644,Cristiano,Christian


In [35]:
#inspecting the list visually
for index, row in df_equal_names.iterrows():
    print('index' + str(index))
    print('Name1:' + str(row["Name1"]) +' Name2:' + str(row["Name2"]))


index98
Name1:Marc-André Name2:Marc
index1260
Name1:Oliver Name2:Olivier
index1569
Name1:Nico Name2:Nicolò
index1662
Name1:Nico Name2:Nicola
index2052
Name1:Jonathan Name2:Jon
index2210
Name1:Jonathan Name2:Johan
index2780
Name1:Antonio Name2:António
index3905
Name1:David Name2:Davide
index4535
Name1:Maximilian Name2:Maxim
index5701
Name1:Aleksandar Name2:Alejandro
index5762
Name1:Aleksandar Name2:Alessandro
index5826
Name1:Aleksandar Name2:Alexander
index5979
Name1:Aleksandar Name2:Alexandru
index5993
Name1:Aleksandar Name2:Oleksandr
index6420
Name1:Robert Name2:Norbert
index7196
Name1:Toni Name2:Tomi
index8303
Name1:Florian Name2:Florin
index8316
Name1:Florian Name2:Florinel
index8833
Name1:Jamal Name2:Jemal
index9136
Name1:Kai Name2:Kamil
index10167
Name1:Leroy Name2:Leo
index10631
Name1:Thomas Name2:Tomas
index10818
Name1:Deniz Name2:Denis
index11469
Name1:Niclas Name2:Nicola
index11498
Name1:Niclas Name2:Niklas
index11508
Name1:Niclas Name2:Nicolas
index11582
Name1:Niclas Name2:Ni

In [37]:
#Writing down manually choosed items from list above to delete from list
indexed_to_delete = [
98
,2210
,6420
,7196
,9136
,10167
,40357
,40407
,40409
,40542
,44093
,45639
,46014
,53953
,58502
,58982
,59428
,84731
,85190
,91266
,91445
,91714
,94062
,99632
]

In [38]:
#deleting items from list of df_equal_names
df_equal_names = df_equal_names.drop(index=indexed_to_delete)

In [41]:
#Creating dictionary where key = Name for replacing and value = new name

#first, creating list of names for looping
lnames_loop = pd.unique(df_equal_names[['Name1', 'Name2']].values.ravel('K'))
                        
#dtoreplace - resuly dictionary
dtoreplace = {}
                        
#fill in dict                        
for i in lnames_loop:
     if dtoreplace.get(i) == None: #if name is already exists, ignore
        dtoreplace[i] = i #replace name to the same name for futher check if the name is already exist
        #ltemp_list - get all names to replace for i
        ltemp_list = pd.unique(df_equal_names[(df_equal_names["Name1"] == i) | (df_equal_names["Name2"] == i)].values.ravel('K')) 
        for j in ltemp_list:
            if dtoreplace.get(j) == None: #if name is already exists, ignore
                print(j + ' replaced by ' + i)
                dtoreplace[j] = i #each name from ltemp_list replacing by i
                        
#as a result we are printing logs of replacements

Olivier replaced by Oliver
Nicolò replaced by Nico
Nicola replaced by Nico
Nicolas replaced by Nico
Nicolae replaced by Nico
Jon replaced by Jonathan
Johan replaced by Jonathan
António replaced by Antonio
Davide replaced by David
Maxim replaced by Maximilian
Alejandro replaced by Aleksandar
Alessandro replaced by Aleksandar
Alexander replaced by Aleksandar
Alexandru replaced by Aleksandar
Oleksandr replaced by Aleksandar
Florin replaced by Florian
Florinel replaced by Florian
Jemal replaced by Jamal
Tomas replaced by Thomas
Denis replaced by Deniz
Niklas replaced by Niclas
Zan replaced by Zander
Jaka replaced by Jack
Granit replaced by Grant
Andrej replaced by Andrew
Andrea replaced by Andrew
Andreas replaced by Andrew
Andrei replaced by Andrew
Gregor replaced by Greg
Kenan replaced by Kenny
Peter replaced by Péter
William replaced by Willi
Loïs replaced by Loïc
Anders replaced by András
Daniel replaced by Dániel
Krzysztof replaced by Krisztofer
Martín replaced by Martin
Marin replaced

In [42]:
# print dictionary 
dtoreplace

{'Oliver': 'Oliver',
 'Olivier': 'Oliver',
 'Nico': 'Nico',
 'Nicolò': 'Nico',
 'Nicola': 'Nico',
 'Nicolas': 'Nico',
 'Nicolae': 'Nico',
 'Jonathan': 'Jonathan',
 'Jon': 'Jonathan',
 'Johan': 'Jonathan',
 'Antonio': 'Antonio',
 'António': 'Antonio',
 'David': 'David',
 'Davide': 'David',
 'Maximilian': 'Maximilian',
 'Maxim': 'Maximilian',
 'Aleksandar': 'Aleksandar',
 'Alejandro': 'Aleksandar',
 'Alessandro': 'Aleksandar',
 'Alexander': 'Aleksandar',
 'Alexandru': 'Aleksandar',
 'Oleksandr': 'Aleksandar',
 'Florian': 'Florian',
 'Florin': 'Florian',
 'Florinel': 'Florian',
 'Jamal': 'Jamal',
 'Jemal': 'Jamal',
 'Thomas': 'Thomas',
 'Tomas': 'Thomas',
 'Deniz': 'Deniz',
 'Denis': 'Deniz',
 'Niclas': 'Niclas',
 'Niklas': 'Niclas',
 'Zander': 'Zander',
 'Zan': 'Zander',
 'Jack': 'Jack',
 'Jaka': 'Jack',
 'Grant': 'Grant',
 'Granit': 'Grant',
 'Andrew': 'Andrew',
 'Andrej': 'Andrew',
 'Andrea': 'Andrew',
 'Andreas': 'Andrew',
 'Andrei': 'Andrew',
 'Greg': 'Greg',
 'Gregor': 'Greg',
 'Joh

In [43]:
# Replacing names in original dataset
df_palyersName['Name'] = df_palyersName['Name'].replace(dtoreplace)

### And finally there are top 5 common names of players at EURO 2024 !

In [47]:
top_5_names = df_palyersName['Name'].value_counts().head(5)
top_5_names

Name
Luka          12
Aleksandar    12
Thomas        11
David         10
Marc           8
Name: count, dtype: int64